In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
pd.options.mode.chained_assignment = None  # default='warn'
import csv
from operator import itemgetter
from sqlalchemy import create_engine
import networkx as nx
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.

# Creacion de un dataframe fitcio

Segmentacion de poblaciones de usuarios (gustos musicales) y artistas (generos musicales)

usuarios:

01  a  10  
11 a   20  
21 a   30  
31 a   40  
41 a   50  
51 a   60  
61 a   70  
71 a   80  
81 a   90  
91 a   100  

aristas  
 
01 a    40  pop  
41 a    80  rock  
81 a    120 jazz  
121 a   160 reggeton  
161 a   200 trap    
201 a   240 cumbia  
 


In [ ]:
usuarios=[]
artistas = []

usuarios que escuchan mayormente pop y algo de rock

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(1,10))
  artistas.append(random.randint(10,50))

 usuarios que escuchan mayormente rock y jazz

In [ ]:
for i in range(1,51):
  usuarios.append(random.randint(11,20))
  artistas.append(random.randint(41,80))  #rock
for i in range(1,51):
  usuarios.append(random.randint(11,20))  
  artistas.append(random.randint(161,200))  # jazz

 usuarios que escuchan mayormente jazz

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(21,30))
  artistas.append(random.randint(161,200))

 usuarios que escuchan mayormente tra, raggeton y algo de cumbia

In [ ]:
for i in range(1,61):
  usuarios.append(random.randint(31,40))
  artistas.append(random.randint(121,160))  # 60% trap
for i in range(1,31):
  usuarios.append(random.randint(31,40))  
  artistas.append(random.randint(81,120))  # 30% reggeton
for i in range(1,11):
  usuarios.append(random.randint(31,40))  
  artistas.append(random.randint(201,240))  # 10% cumbia

 usuarios que escuchan solo cumbia

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(41,50))  
  artistas.append(random.randint(201,240))  # 100% cumbia

 usuarios que escuchan de todo

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(51,60))
  artistas.append(random.randint(1,240))

 usuarios que escuchan de todo

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(61,70))
  artistas.append(random.randint(1,240))

usuarios que escuchan de todo

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(71,80))
  artistas.append(random.randint(1,240))

 usuarios que escuchan rock y trap

In [ ]:
for i in range(1,71):
  usuarios.append(random.randint(81,90))
  artistas.append(random.randint(41,80))
for i in range(1,31):
  usuarios.append(random.randint(81,90))
  artistas.append(random.randint(121,160))

 usuarios que escuchan mayormente pop 

In [ ]:
for i in range(1,101):
  usuarios.append(random.randint(91,100))
  artistas.append(random.randint(1,40))

In [ ]:
df_origen= pd.DataFrame({'users':usuarios,
                  'artist':artistas})


In [ ]:
df_origen

In [ ]:
df_origen.describe()


un poco de limpieza. artistas unicos

In [ ]:
df_origen.drop_duplicates(subset=['users', 'artist'], keep='first',inplace=True)

In [ ]:
len(df_origen)

exportamos csv para alimentar una bbdd postegres

In [ ]:
df_origen.to_csv('df_origen.csv', index=False)

## conexion con bbdd postgres

armo un dataset a partir de tabla de postgres



In [3]:
# follows django database settings format, replace with your own settings
DATABASES = {
    'Musicbrainz':{
        'NAME': 'Musicbrainz',
        'USER': 'postgres',
        'PASSWORD': 'qweasd',
        'HOST': 'localhost',
        'PORT': 5432,
    },
}




In [4]:
# choose the database to use
db = DATABASES['Musicbrainz']



In [5]:
# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

In [6]:
# create sqlalchemy engine
engine = create_engine(engine_string)

In [12]:
# read a table from database into pandas dataframe, replace "tablename" with your table name
df_origen = pd.read_sql_table('artistcredit',engine)

In [13]:
df_origen

,id,name,artist_count,ref_count,created
0,202094,Ani & Beau,1,6,2011-05-16 16:32:11.963929+00:00
1,288788,Rouse,1,90,2011-05-16 16:32:11.963929+00:00
2,485926,The Ascient Rebels,1,2,2011-05-16 16:32:11.963929+00:00
3,809019,Toni Childs & Phil Ramacom,2,2,2011-05-16 16:32:11.963929+00:00
4,218132,Londonairs,1,2,2011-05-16 16:32:11.963929+00:00
...,...,...,...,...,...
1188704,1602918,Armin van Buuren vs Sophie Ellis-Bextor,2,12,2016-01-30 11:00:48.712136+00:00
1188705,1821674,김희철 x 민경훈,2,1,2016-12-24 00:12:45.248979+00:00
1188706,3415,Ray Charles,1,14486,2011-05-16 16:32:11.963929+00:00
1188707,663697,The Hall Effect,1,60,2011-05-16 16:32:11.963929+00:00


transformamos un poco el dataframe para poder alimentar un algoritmo de clusterizacion

In [ ]:
print(df_origen['artist'].sort_values().unique())

In [ ]:
print(df_origen['artist'].value_counts())

In [ ]:
intervalos=range(1,max(df_origen.artist)+2,1)

In [ ]:
intervalos

In [ ]:
sns.histplot(data=df_origen, x="artist",bins=intervalos)

Matriz de matcheo

In [ ]:
df1=df_origen
df2=df_origen
df3 = pd.merge(df1, df2, on='artist')


In [ ]:
df1[df1.users==1]

In [ ]:
df_std = df3.groupby(['users_x', 'users_y']).agg({'artist': ['count']})
df_std=df_std.reset_index()
df_std

In [ ]:
user_1=df_std.users_x
user_2=df_std.users_y
match=df_std[('artist','count')]

In [ ]:
df_match=pd.DataFrame()
df_match['user_1']=user_1
df_match['user_2']=user_2
df_match['match']=match

In [ ]:
df_match

In [ ]:
m=df_match.pivot(index='user_1', columns='user_2', values='match')
m.fillna(0, inplace=True)
m

In [ ]:
t=np.arange(1,101,2)
t

In [ ]:
plt.figure(figsize=(20,16))


g=sns.scatterplot(data=df_match, x="user_1", y="user_2", hue='match', size="match",
    sizes=(50, 300),legend="full")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.xticks(t)
plt.yticks(t)


## Algoritmos de clustering

### Hacemos un one hot encoder sobre la feature artista

In [ ]:
data_encoded = pd.get_dummies(df_origen, columns = ['artist'])
data_encoded

In [ ]:
df4 = data_encoded.groupby(['users'])[ data_encoded.columns[1:].values].sum()
df4=df4.reset_index()

In [ ]:
df4

In [ ]:
df4[df4.users==1].values.sum()

In [ ]:
df4.iloc[1:10,:20]

Aplicacion del modelo

In [ ]:
X=df4.iloc[:,1:]
y=df4.iloc[:,0]

In [ ]:
y

k Means

In [ ]:
# Importmaos la libreria
from sklearn.cluster import KMeans

# Especificamos el numero adecuado de clusters en cada caso
kmeans_1 = KMeans(n_clusters=4)


3. Entrenamos los modelos con los datos

In [ ]:
kmeans_1.fit(X)


Luego de entrenar los modelos, podemos consultar las etiquetas que se le asignó a cada instancia y el lugar de los centroides.

In [ ]:
etiquetas_1 = kmeans_1.labels_
#print(etiquetas_1.shape)
etiquetas_1

In [ ]:
cluster_map = pd.DataFrame()
cluster_map['user'] = y
cluster_map['cluster_KMeans'] = kmeans_1.labels_

In [ ]:
cluster_map

In [ ]:
#centros_1 = kmeans_1.cluster_centers_
#print(centros_1)

## DBscan

In [ ]:
from sklearn.cluster import DBSCAN

BUSQUEDA DE LOS MEJORES HIPERPARAMETROS PARA DBSCAN

In [ ]:
from itertools import product

eps_values = np.arange(2,2.5,.1) # eps values to be investigated
min_samples = np.arange(1,3) # min_samples values to be investigated

DBSCAN_params = list(product(eps_values, min_samples))

In [ ]:
DBSCAN_params

In [ ]:
from sklearn.metrics import silhouette_score

no_of_clusters = []
sil_score = []

for p in DBSCAN_params:
    DBS_clustering = DBSCAN(eps=p[0], min_samples=p[1]).fit(X)
    no_of_clusters.append(len(np.unique(DBS_clustering.labels_)))
    sil_score.append(silhouette_score(X, DBS_clustering.labels_))

In [ ]:
tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
tmp['No_of_clusters'] = no_of_clusters

pivot_1 = pd.pivot_table(tmp, values='No_of_clusters', index='Min_samples', columns='Eps')

fig, ax = plt.subplots(figsize=(12,6))
sns.heatmap(pivot_1, annot=True,annot_kws={"size": 16}, cmap="YlGnBu", ax=ax)
ax.set_title('Number of clusters')
plt.show()

In [ ]:
tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
tmp['Sil_score'] = sil_score

pivot_1 = pd.pivot_table(tmp, values='Sil_score', index='Min_samples', columns='Eps')

fig, ax = plt.subplots(figsize=(18,6))
sns.heatmap(pivot_1, annot=True, annot_kws={"size": 10}, cmap="YlGnBu", ax=ax)
plt.show()

In [ ]:
# En este paso tenemos que definir el epsilon y en min_samples, los parametros del método.
db_1 = DBSCAN(eps=3, min_samples=3)
db_1.fit(X)
db_1.labels_

In [ ]:
cluster_map['cluster_DBSCAN'] = db_1.labels_
cluster_map


In [ ]:
db_1.labels_

In [ ]:
m

Algortitmo affinity propagation 

In [ ]:
from sklearn.cluster import AffinityPropagation

In [ ]:
'''no_of_clusters = []
preferences = range(-5,-5,1) # arbitraty chosen range
af_sil_score = [] # silouette scores

for p in preferences:
    AF = AffinityPropagation(preference=p, max_iter=200).fit(X)
    no_of_clusters.append((len(np.unique(AF.labels_))))
    af_sil_score.append(silhouette_score(X, AF.labels_))
    
af_results = pd.DataFrame([preferences, no_of_clusters, af_sil_score], index=['preference','clusters', 'sil_score']).T
af_results.sort_values(by='sil_score', ascending=False).head() # display only 5 best scores
'''

In [ ]:
 AP_1 = AffinityPropagation(preference=-800 max_iter=200).fit(X)

In [ ]:
cluster_map['cluster_AP'] = AP_1.labels_
cluster_map


# Grafo

In [ ]:
import networkx as nx

In [ ]:
df_relations=df_match[df_match['user_1']<df_match['user_2']]
df_relations.reset_index(inplace=True)

In [ ]:
df_relations

en 100 usuario las posibles combinaciones de a pares son 100C2 4950 

In [ ]:
G = nx.from_pandas_edgelist(
    df_relations,
    source='user_1',
    target='user_2',
    #edge_labels={(source, target): match}
    #edge_key= "my_edge_key",
    #edge_attr=['match'],
    #create_using=nx.MultiGraph(),
)

In [ ]:
plt.figure(figsize=(15,12))
nx.draw_kamada_kawai(G, node_size=4000,with_labels=True, font_size=25, font_color="orange", font_weight="bold")

In [ ]:
plt.figure(figsize=(15,12))
nx.draw_spring(G, node_size=4000,with_labels=True, font_size=25, font_color="orange", font_weight="bold")

In [ ]:
print(nx.info(G))

In [ ]:
G_weighted = nx.Graph()

In [ ]:
for i in range(0, len(df_relations)):
  G_weighted.add_edge(df_relations.iloc[i,1],  df_relations.iloc[i,2],   weight=df_relations.iloc[i,3])
  

In [ ]:
df_relations.iloc[:,3].describe()


In [ ]:
elarge = [(u, v) for (u, v, d) in G_weighted.edges(data=True) if d['weight'] >3 ]
emedium = [(u, v) for (u, v, d) in G_weighted.edges(data=True) if (d['weight'] >=2 and d['weight'] <3) ]
esmall = [(u, v) for (u, v, d) in G_weighted.edges(data=True) if d['weight'] <2]
plt.figure(figsize=(20,20))
pos = nx.circular_layout(G_weighted)  # positions for all nodes

In [ ]:
plt.figure(figsize=(20,20))
# nodes

nx.draw_networkx_nodes(G_weighted, pos, node_size=300)

# edges

nx.draw_networkx_edges(G_weighted, pos, edgelist=elarge,width=3, alpha=1, edge_color='r', style='solid')

nx.draw_networkx_edges(G_weighted, pos, edgelist=emedium,width=2, alpha=0.5, edge_color='g', style='dotted')

nx.draw_networkx_edges(G_weighted, pos, edgelist=esmall,width=1, alpha=0.3, edge_color='b', style='dashed')

# labels

nx.draw_networkx_labels(G_weighted, pos, font_size=20, font_family='sans-serif')

plt.axis('off')

plt.show()


In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx(G_weighted);

In [ ]:
pos = nx.spring_layout(G_weighted,threshold=0.0001,k=0.92,dim=2,scale=.0005)
betCent = nx.betweenness_centrality(G_weighted, normalized=True, endpoints=True)
node_color = [20000.0 * G_weighted.degree(v) for v in G_weighted]
node_size =  [v*v * 1000000 for v in betCent.values()]
plt.figure(figsize=(20,20))
nx.draw_networkx(G_weighted, pos=pos, with_labels=True,
                 node_color=node_color,
                 node_size=node_size )
plt.axis('off');

In [ ]:
pos = nx.spring_layout(G_weighted.degree)
pos

Algunas métricas

In [ ]:
nx.clustering(G_weighted,9)

In [ ]:
nx.average_clustering(G_weighted)

In [ ]:
nx.degree_centrality(G_weighted)

In [ ]:
nx.eigenvector_centrality(G_weighted)

In [ ]:
nx.closeness_centrality(G_weighted)

In [ ]:
nx.betweenness_centrality(G_weighted)

In [ ]:
G = nx.complete_graph(4)
pos = nx.nx_pydot.graphviz_layout(G)
pos = nx.nx_pydot.graphviz_layout(G, prog="dot")